In [1]:
import findspark
findspark.init()
from pyspark import SparkContext as sc

import os

from pyspark.sql import SQLContext, Row

import pyspark

import sys
import string
from csv import reader
from functools import reduce
from pyspark.sql import functions as f
from collections import defaultdict
import datetime
from tableCollections import TableCollections
import os

In [2]:
data_path = ['data']
parking_filepath = os.sep.join(data_path + ['parking.csv'])

sc = sc(appName="Parking")
sqlContext = SQLContext(sc)

In [3]:
parkingTable = sqlContext.read.format('csv').options(header='true',inferschema='true').load('data/parking.csv')

In [4]:
openTable = sqlContext.read.format('csv').options(header='true',inferschema='true').load('data/open-parking.csv')

In [5]:
tc = TableCollections(sqlContext, sc)

In [6]:
tc.register(parkingTable, "parking")

Num Function


In [ ]:
tc.timeColWithinRange(self, minTime, maxTime)

In [ ]:
tc.numColWithinRange(self, minNum, maxNum)

In [ ]:
tc.getNumRange(self,colList)

In [ ]:
tc.getTimeRange(self, colList)

In [ ]:
tc.getSimilarNumCols(self, tableColName, threshold=0)

In [ ]:
tc.returnIntersecWithinCols(self,colList)

In [ ]:
tc.frequentVals(self, colList, topN)

In [ ]:
tc.returnOutliers(self, colList, percentage)

In [ ]:
tc.colsWithAndWithout(self, colList, withList, withoutList)

In [ ]:
tc.getCardinality(self, colList)

In [ ]:
tc.getColsOfDatatype(self, df, coltype)

In [ ]:
tc.colsNameSimilarity(self, df, category = None, df2=None)

In [ ]:
tc.getColsofCategory(self, tableName, colList, category)

In [ ]:
tc.countNullValues(self, df)

In [ ]:
tc.returnUnionWithinCols(self,colList)